In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv

swh571.csv  swh573.csv	swh712.csv  swh714.csv	swh716.csv  swh718.csv
swh572.csv  swh711.csv	swh713.csv  swh715.csv	swh717.csv


In [ ]:
%cd /content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv

/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv


In [ ]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import math
import scipy.stats as stats 
from numpy import array
from pandas import read_csv
from math import sqrt
from numpy import split
from plotly.subplots import make_subplots
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from datetime import datetime
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## Visualisasi Data Mentah

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/swh/csv/swh571.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# print(data)

# kolom_interpolasi = ['swh']
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')


# data['swh'] = imputer.fit_transform(data['swh'].values.reshape(-1, 1))


# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

# data = preparedata(daerah)

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude  expver                 time       swh
0          100.5       5.0     NaN  2022-01-01 00:00:00  0.114618
1          100.5       5.0     NaN  2022-01-01 01:00:00  0.115161
2          100.5       5.0     NaN  2022-01-01 02:00:00  0.115229
3          100.5       5.0     NaN  2022-01-01 03:00:00  0.114753
4          100.5       5.0     NaN  2022-01-01 04:00:00  0.113804
...          ...       ...     ...                  ...       ...
14571      100.5       5.0     5.0  2023-05-02 01:00:00  0.087120
14572      100.5       5.0     5.0  2023-05-02 02:00:00  0.086414
14573      100.5       5.0     1.0  2023-05-02 02:00:00  0.088454
14574      100.5       5.0     5.0  2023-05-02 03:00:00  0.090495
14575      100.5       5.0     1.0  2023-05-02 03:00:00  0.090495

[14576 rows x 5 columns]


In [ ]:

# Menampilkan grafik
plt.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=data['time'], y=data['swh'],
                         mode='lines', name='Data SWH'))

# Edit the layout
fig.update_layout(xaxis_title='<b>Date</b>', yaxis_title='<b><i>Hs</i> [m]</b>')

fig.show()

## Simple Model

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.metrics import Accuracy

# Load and preprocess the data
swh = data["swh"].values.reshape(-1, 1)  # Reshape to a column vector
scaler = MinMaxScaler()
swh_scaled = scaler.fit_transform(swh)

# Split the data into training and testing sets
train_size = int(len(swh_scaled) * 0.8)
train_data = swh_scaled[:train_size]
test_data = swh_scaled[train_size:]

# Define the function to create the input sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)

# Set the sequence length and create the input sequences
seq_length = 10  # Adjust as needed
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(seq_length, 1)),
    tf.keras.layers.Dense(1)
])

# Create an instance of the Accuracy metric
# accuracy_metric = Accuracy()

# Compile the model and add the accuracy metric
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

# Evaluate the model
train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)

# Make predictions
predictions = model.predict(X_test)

# Inverse scale the predictions and actual values
predictions = scaler.inverse_transform(predictions)
actual_values = scaler.inverse_transform(y_test)



Epoch 1/10
365/365 - 4s - loss: 0.4561 - accuracy: 1.7167e-04 - 4s/epoch - 12ms/step
Epoch 2/10
365/365 - 3s - loss: 0.4545 - accuracy: 1.7167e-04 - 3s/epoch - 9ms/step
Epoch 3/10
365/365 - 4s - loss: 0.4546 - accuracy: 1.7167e-04 - 4s/epoch - 10ms/step
Epoch 4/10
365/365 - 2s - loss: 0.4531 - accuracy: 1.7167e-04 - 2s/epoch - 6ms/step
Epoch 5/10
365/365 - 2s - loss: 0.4530 - accuracy: 1.7167e-04 - 2s/epoch - 5ms/step
Epoch 6/10
365/365 - 2s - loss: 0.4531 - accuracy: 1.7167e-04 - 2s/epoch - 5ms/step
Epoch 7/10
365/365 - 2s - loss: 0.4529 - accuracy: 1.7167e-04 - 2s/epoch - 5ms/step
Epoch 8/10
365/365 - 2s - loss: 0.4529 - accuracy: 1.7167e-04 - 2s/epoch - 5ms/step
Epoch 9/10
365/365 - 3s - loss: 0.4530 - accuracy: 1.7167e-04 - 3s/epoch - 8ms/step
Epoch 10/10
365/365 - 3s - loss: 0.4529 - accuracy: 1.7167e-04 - 3s/epoch - 7ms/step
91/91 [==============================] - 1s 3ms/step


In [ ]:
# Calculate RMSE
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('rmse:',rmse)
# print("Accuracy: {:.2f}%".format(accuracy * 100))
# Print RMSE
# print("RMSE: {:.2f}".format(rmse))
# # Print the predictions and actual values
# for i in range(len(predictions)):
#     print("Predicted:", predictions[i][0], "Actual:", actual_values[i][0])

91/91 [==============================] - 0s 3ms/step
rmse: 0.04841270343417196


In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the predicted values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test))),
    y=y_pred.flatten(),
    mode='lines',
    name='Predicted Values'
))

# Add the actual values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test))),
    y=y_test.flatten(),
    mode='lines',
    name='Actual Values'
))

# Set the layout
fig.update_layout(
    title='Predicted Values vs Actual Values',
    xaxis_title='Time',
    yaxis_title='Value',
)

# Show the figure
fig.show()


## Complex Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import plotly.graph_objs as go
import plotly.io as pio

# Load the data
# data = pd.read_csv('swh_data.csv')

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['swh'].values.reshape(-1, 1))

# Split the data into train and test sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Function to create sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Set the sequence length
seq_length = 60

# Create the training sequences
X_train, y_train = create_sequences(train_data, seq_length)

# Create the test sequences
X_test, y_test = create_sequences(test_data, seq_length)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=2)





Epoch 1/10
363/363 - 87s - loss: 0.4721 - accuracy: 8.6207e-05 - val_loss: 0.4357 - val_accuracy: 0.0000e+00 - 87s/epoch - 241ms/step
Epoch 2/10
363/363 - 38s - loss: 0.4600 - accuracy: 1.7241e-04 - val_loss: 0.4333 - val_accuracy: 0.0000e+00 - 38s/epoch - 104ms/step
Epoch 3/10
363/363 - 39s - loss: 0.4577 - accuracy: 1.7241e-04 - val_loss: 0.4326 - val_accuracy: 0.0000e+00 - 39s/epoch - 108ms/step
Epoch 4/10
363/363 - 36s - loss: 0.4568 - accuracy: 1.7241e-04 - val_loss: 0.4334 - val_accuracy: 0.0000e+00 - 36s/epoch - 98ms/step
Epoch 5/10
363/363 - 49s - loss: 0.4568 - accuracy: 1.7241e-04 - val_loss: 0.4329 - val_accuracy: 0.0000e+00 - 49s/epoch - 136ms/step
Epoch 6/10
363/363 - 43s - loss: 0.4564 - accuracy: 1.7241e-04 - val_loss: 0.4322 - val_accuracy: 0.0000e+00 - 43s/epoch - 118ms/step
Epoch 7/10
363/363 - 39s - loss: 0.4558 - accuracy: 1.7241e-04 - val_loss: 0.4322 - val_accuracy: 0.0000e+00 - 39s/epoch - 107ms/step
Epoch 8/10
363/363 - 38s - loss: 0.4556 - accuracy: 1.7241e-04 

In [ ]:
# Generate predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse scale the predictions
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print(f"Train RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")

90/90 [==============================] - 3s 30ms/step
Train RMSE: 0.01
Test RMSE: 0.01


In [ ]:
# Create a figure for visualization
fig = go.Figure()

# Add the actual values trace
fig.add_trace(go.Scatter(
    x=data.index[:train_size],
    y=data['swh'][:train_size],
    name='Actual (Train)',
    mode='lines',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=data.index[train_size:],
    y=data['swh'][train_size:],
    name='Actual (Test)',
    mode='lines',
    line=dict(color='green')
))

# Add the predicted values trace
fig.add_trace(go.Scatter(
    x=data.index[seq_length:seq_length+len(train_predictions)],
    y=train_predictions.flatten(),
    name='Predicted (Train)',
    mode='lines',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=data.index[seq_length+train_size:seq_length+train_size+len(test_predictions)],
    y=test_predictions.flatten(),
    name='Predicted (Test)',
    mode='lines',
    line=dict(color='orange')
))

# Set the layout
fig.update_layout(
    title='SWH Prediction',
    xaxis_title='Date',
    yaxis_title='SWH',
    legend=dict(x=0, y=1),
)

# Save and display the figure
pio.show(fig)

# Fungsi


## splitting data

In [ ]:
# convert data into x and y 
def change_data(train, n_input, n_out=3):
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	for _ in range(n_input, len(data)):
		in_end = in_start + n_input
		out_end = in_end + n_out
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		in_start += 1
	return array(X), array(y)

In [ ]:
def splitting_data(data):
  # end_data = list(data['time']).index(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S' ))
  data= data.loc[:,['swh']]
  data= data.reset_index(drop=True)
 
  # scaler_label = StandardScaler().fit(data['swh'].values.reshape(-1, 1))
  # scaler_label = scaler_label.transform(data['swh'].values.reshape(-1, 1))

  scaler_label = StandardScaler().fit(data['swh'].values.reshape(-1, 1))
  scaler_label = scaler_label.transform(data['swh'].values.reshape(-1, 1))

  print(scaler_label)
  data_new = pd.DataFrame(scaler_label,index = data.index, columns=data.columns)
  train = data_new[:round(len(data_new)*0.8)]
  val = data_new[round(len(data_new)*0.8) : round(len(data_new)*0.9) ]
  test = data_new[round(len(data_new)*0.9):]

  i = 0
  while len(train) % 24 != 0:
    train = train[:round(len(train)*0.8) + i]
    i += 1
  j = 0
  while len(test) % 24 != 0:
    test = test[round(len(test)*0.9) - j:]
    j += 1
  
  val = val[round(len(val)*0.8)+i : round(len(val)*0.9)-j]
  while len(val) % 24 != 0:
    i -= 1
    val = val[round(len(val)*0.8)+i : round(len(val)*0.9)-j]


  # resturvture into windows of 24 hours data
  train = array(split(train, round(len(train)/24)))
  val = array(split(val, round(len(val)/24)))
  test = array(split(test, round(len(test)/24)))

  n_input,n_out = 36,24

  n_steps_in, n_steps_out = 36,24
  X_train, y_train =  change_data(train,n_steps_in, n_steps_out)
  X_val, y_val =  change_data(val,n_steps_in, n_steps_out)
  
  return X_train, y_train, X_val, y_val, train, val, test ,j

## model 

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True    

def define_model(unit1,unit2):
  tf.keras.backend.clear_session()
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
  model1 = Sequential()
  model1.add(LSTM(unit1, activation='relu', input_shape=(n_timesteps, n_features)))
  model1.add((Dense(unit2, activation='relu')))
  model1.add((Dense(n_outputs)))

  # cp1 = ModelCheckpoint('model1/', save_best_only=True)
  model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), 
                 metrics=['accuracy'])
    
  return model1

def fit_model(X_train, y_train, X_val, y_val):
  # Instantiate the callback
  callbacks = myCallback()
  
  # fitting model 
  history = model1.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=10, callbacks=callbacks,verbose=2)
  # EXERCISE: Save the trained model as a Keras HDF5 file. 
  # saved_model_path = '/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/o2/model/'+daerah+'.h5'
  # YOUR CODE HERE
  
  # model1.save(saved_model_path)
  return history

In [ ]:
def forecast(data, daerah):
    scaler_label = StandardScaler().fit(data['swh'].values.reshape(-1, 1))
    data_new = data

    history = [x for x in val]
    predictions = []
    n_input = 24
    for k in range(len(test)):
        datax = np.array(history)
        datax = datax.reshape((datax.shape[0] * datax.shape[1], datax.shape[2]))
        input_x = datax[-n_input:, :]
        input_x = input_x.reshape((1, len(input_x), 1))
        # predict for 24 hours ahead
        yhat = model1.predict(input_x, verbose=0)
        yhat_sequence = yhat[0]
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history
        history.append(test[k, :])

    predictions = np.array(predictions)
    predictions = scaler_label.inverse_transform(predictions)

    observasi = test
    observasi = test[:, :, 0]
    observasi = scaler_label.inverse_transform(observasi)

    predictions = predictions.reshape((predictions.shape[0] * predictions.shape[1], 1))
    observasi = observasi.reshape((observasi.shape[0] * observasi.shape[1], 1))

    test_results = pd.DataFrame(data={'Predictions': predictions.flatten(), 'Actuals': observasi.flatten()})
    test_results['Date'] = data_new['time'].iloc[round(len(data_new) * 0.9) - j:].values

    return test_results

def forecast_viz(test_results):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Predictions'],
                             mode='lines',
                             name='Fitting Data Testing'))
    fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Actuals'],
                             mode='lines',
                             name='Observation'))

    # Edit the layout
    fig.update_layout(xaxis_title='<b>Date</b>', yaxis_title='<b><i>Hs</i> [m]</b>')

    return fig.show()

# # Usage example
# test_results = forecast(data, 'WPP 571')
# forecast_viz(test_results)


In [ ]:
def forecast(data,daerah):

  scaler_label = StandardScaler().fit(data['swh'].values.reshape(-1, 1))
  # scaler_label = scaler_label.transform(data['chl'].values.reshape(-1, 1))
  data_new = data

  history = [x for x in val]
  predictions = list()
  n_input = 9
  for k in range(len(test)):
    datax = array(history)
    datax = datax.reshape((datax.shape[0]*datax.shape[1], datax.shape[2]))
    input_x = datax[-n_input:, :]
    input_x = input_x.reshape((1, len(input_x), 1))
    # predict the week
    yhat = model1.predict(input_x, verbose=0)
    yhat_sequence = yhat[0]
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next week
    history.append(test[k, :])

  prediksi = array(predictions)
  prediksi = scaler_label.inverse_transform(predictions)
  
  observasi = test
  
  observasi = test[:, :, 0]
  observasi = scaler_label.inverse_transform(observasi)
  
  prediksi = prediksi.reshape((prediksi.shape[0]*prediksi.shape[1], 1))
  observasi = observasi.reshape((observasi.shape[0]*observasi.shape[1], 1))
  print(len(prediksi))
  print(len(observasi))
  print(prediksi.shape, observasi.shape)
  test_results = pd.DataFrame(data={'Predictions':prediksi.flatten(), 
                                   'Actuals':observasi.flatten()})
  
  test_results['Date'] = data_new['time'].iloc[round(len(data_new)*0.9) - j:].values
  # test_results.to_csv('/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/chl/prediksi/'+daerah+'.csv')
  return test_results
  
def forecast_viz():
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Predictions'],
                    mode='lines',
                    name='Predictions'))
  fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Actuals'],
                    mode='lines',
                    name='Observation'))
  
  # Edit the layout
  fig.update_layout( xaxis_title='<b>Date</b>',
                   yaxis_title='<b><i>Hs</i> [m]</b>')
  
  return fig.show()

## Evaluating Model

In [ ]:
def evaluate():
  MSE = mean_squared_error(test_results['Actuals'],test_results['Predictions'])
  RMSE = math.sqrt(MSE)
  corr, _ = pearsonr(test_results['Actuals'],test_results['Predictions'])

  gap_sum = 0
  for j in range(len(test_results['Actuals'])):
    gap = test_results['Predictions'][j] - test_results['Actuals'][j]
    gap_sum += gap
    
  bias = gap_sum/len(test_results['Actuals'])
  _, pvalue = stats.ttest_ind(test_results['Actuals'],test_results['Predictions'], equal_var=True)

  value = [round(RMSE,5),round(corr,5), round(bias,5), round(pvalue,5)]
  eval = pd.DataFrame(value, index =['rmse', 'corr', 'bias','pvalue'],
                                              columns =['values'])
  return eval

# WPP571

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh571.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

       longitude  latitude  expver                 time       swh
0          100.5       5.0     NaN  2022-01-01 00:00:00  0.114618
1          100.5       5.0     NaN  2022-01-01 01:00:00  0.115161
2          100.5       5.0     NaN  2022-01-01 02:00:00  0.115229
3          100.5       5.0     NaN  2022-01-01 03:00:00  0.114753
4          100.5       5.0     NaN  2022-01-01 04:00:00  0.113804
...          ...       ...     ...                  ...       ...
14571      100.5       5.0     5.0  2023-05-02 01:00:00  0.087120
14572      100.5       5.0     5.0  2023-05-02 02:00:00  0.086414
14573      100.5       5.0     1.0  2023-05-02 02:00:00       NaN
14574      100.5       5.0     5.0  2023-05-02 03:00:00  0.090495
14575      100.5       5.0     1.0  2023-05-02 03:00:00       NaN

[14576 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude  expver                 time       swh
0          100.5       5.0     NaN  2022-01-01 00:00:00  0.114618
1          100.5       5.0     NaN  2022-01-01 01:00:00  0.115161
2          100.5       5.0     NaN  2022-01-01 02:00:00  0.115229
3          100.5       5.0     NaN  2022-01-01 03:00:00  0.114753
4          100.5       5.0     NaN  2022-01-01 04:00:00  0.113804
...          ...       ...     ...                  ...       ...
14571      100.5       5.0     5.0  2023-05-02 01:00:00  0.087120
14572      100.5       5.0     5.0  2023-05-02 02:00:00  0.086414
14573      100.5       5.0     1.0  2023-05-02 02:00:00  0.088454
14574      100.5       5.0     5.0  2023-05-02 03:00:00  0.090495
14575      100.5       5.0     1.0  2023-05-02 03:00:00  0.090495

[14576 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.96351251]
 [-0.95898116]
 [-0.95841399]
 ...
 [-1.18189861]
 [-1.16486762]
 [-1.16486762]]


In [ ]:
train.shape, val.shape, test.shape

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

In [ ]:
test_results = forecast(data,'WPP571')
test_results

In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp571'+'.h5'
model1.save(saved_model_path)

# WPP572

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh572.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
train.shape, val.shape, test.shape

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

In [ ]:
test_results = forecast(data,'WPP572')
test_results

In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp572'+'.h5'
model1.save(saved_model_path)

# WPP573

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh573.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
train.shape, val.shape, test.shape

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

In [ ]:
test_results = forecast(data,'WPP573')
test_results

In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp573'+'.h5'
model1.save(saved_model_path)

# WPP711

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh711.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

       longitude  latitude  expver                 time       swh
0          108.5      -0.5     NaN  2022-01-01 00:00:00  0.463241
1          108.5      -0.5     NaN  2022-01-01 01:00:00  0.468195
2          108.5      -0.5     NaN  2022-01-01 02:00:00  0.478983
3          108.5      -0.5     NaN  2022-01-01 03:00:00  0.490721
4          108.5      -0.5     NaN  2022-01-01 04:00:00  0.502731
...          ...       ...     ...                  ...       ...
14571      108.5      -0.5     5.0  2023-05-02 01:00:00  0.222807
14572      108.5      -0.5     5.0  2023-05-02 02:00:00  0.220688
14573      108.5      -0.5     1.0  2023-05-02 02:00:00       NaN
14574      108.5      -0.5     5.0  2023-05-02 03:00:00  0.218648
14575      108.5      -0.5     1.0  2023-05-02 03:00:00       NaN

[14576 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude  expver                 time       swh
0          108.5      -0.5     NaN  2022-01-01 00:00:00  0.463241
1          108.5      -0.5     NaN  2022-01-01 01:00:00  0.468195
2          108.5      -0.5     NaN  2022-01-01 02:00:00  0.478983
3          108.5      -0.5     NaN  2022-01-01 03:00:00  0.490721
4          108.5      -0.5     NaN  2022-01-01 04:00:00  0.502731
...          ...       ...     ...                  ...       ...
14571      108.5      -0.5     5.0  2023-05-02 01:00:00  0.222807
14572      108.5      -0.5     5.0  2023-05-02 02:00:00  0.220688
14573      108.5      -0.5     1.0  2023-05-02 02:00:00  0.219668
14574      108.5      -0.5     5.0  2023-05-02 03:00:00  0.218648
14575      108.5      -0.5     1.0  2023-05-02 03:00:00  0.218648

[14576 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.09617611]
 [-0.07882238]
 [-0.04102419]
 ...
 [-0.94955522]
 [-0.95312954]
 [-0.95312954]]


((11650, 9, 1), (11650, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3887, 3, 1), (5, 3, 1), (486, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
365/365 - 19s - loss: 0.6995 - accuracy: 0.3888 - val_loss: 0.0048 - val_accuracy: 1.0000 - 19s/epoch - 52ms/step
Epoch 2/10
365/365 - 13s - loss: 0.0112 - accuracy: 0.4015 - val_loss: 0.0017 - val_accuracy: 1.0000 - 13s/epoch - 35ms/step
Epoch 3/10
365/365 - 11s - loss: 0.0105 - accuracy: 0.4602 - val_loss: 0.0025 - val_accuracy: 1.0000 - 11s/epoch - 30ms/step
Epoch 4/10
365/365 - 9s - loss: 0.0101 - accuracy: 0.4750 - val_loss: 0.0030 - val_accuracy: 1.0000 - 9s/epoch - 26ms/step
Epoch 5/10
365/365 - 11s - loss: 0.0097 - accuracy: 0.5256 - val_loss: 0.0051 - val_accuracy: 1.0000 - 11s/epoch - 30ms/step
Epoch 6/10
365/365 - 11s - loss: 0.0096 - accuracy: 0.5364 - val_loss: 0.0028 - val_accuracy: 0.2500 - 11s/epoch - 30ms/step
Epoch 7/10
365/365 - 11s - loss: 0.0092 - accuracy: 0.5441 - val_loss: 0.0098 - val_accuracy: 1.0000 - 11s/epoch - 30ms/step
Epoch 8/10
365/365 - 9s - loss: 0.0096 - accuracy: 0.5429 - val_loss: 6.3044e-04 - val_accuracy: 1.0000 - 9s/epoch - 26ms/step


In [ ]:
test_results = forecast(data,'WPP711')
test_results

1458
1458
(1458, 1) (1458, 1)


,Predictions,Actuals,Date
0,0.246605,0.231910,2023-04-01 19:00:00
1,0.250099,0.231440,2023-04-01 19:00:00
2,0.251493,0.232303,2023-04-01 20:00:00
3,0.233100,0.233166,2023-04-01 20:00:00
4,0.238229,0.233951,2023-04-01 21:00:00
...,...,...,...
1453,0.228065,0.222807,2023-05-02 01:00:00
1454,0.228983,0.220688,2023-05-02 02:00:00
1455,0.223584,0.219668,2023-05-02 02:00:00
1456,0.224848,0.218648,2023-05-02 03:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.01342
corr,0.99187
bias,0.00487
pvalue,0.16750


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp711'+'.h5'
model1.save(saved_model_path)

# WPP712

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh712.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

       longitude  latitude  expver                 time       swh
0          113.0      -6.0     NaN  2022-01-01 00:00:00  0.608309
1          113.0      -6.0     NaN  2022-01-01 01:00:00  0.609734
2          113.0      -6.0     NaN  2022-01-01 02:00:00  0.594942
3          113.0      -6.0     NaN  2022-01-01 03:00:00  0.573026
4          113.0      -6.0     NaN  2022-01-01 04:00:00  0.550296
...          ...       ...     ...                  ...       ...
14571      113.0      -6.0     5.0  2023-05-02 01:00:00  0.261183
14572      113.0      -6.0     5.0  2023-05-02 02:00:00  0.280252
14573      113.0      -6.0     1.0  2023-05-02 02:00:00       NaN
14574      113.0      -6.0     5.0  2023-05-02 03:00:00  0.315567
14575      113.0      -6.0     1.0  2023-05-02 03:00:00       NaN

[14576 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude  expver                 time       swh
0          113.0      -6.0     NaN  2022-01-01 00:00:00  0.608309
1          113.0      -6.0     NaN  2022-01-01 01:00:00  0.609734
2          113.0      -6.0     NaN  2022-01-01 02:00:00  0.594942
3          113.0      -6.0     NaN  2022-01-01 03:00:00  0.573026
4          113.0      -6.0     NaN  2022-01-01 04:00:00  0.550296
...          ...       ...     ...                  ...       ...
14571      113.0      -6.0     5.0  2023-05-02 01:00:00  0.261183
14572      113.0      -6.0     5.0  2023-05-02 02:00:00  0.280252
14573      113.0      -6.0     1.0  2023-05-02 02:00:00  0.297910
14574      113.0      -6.0     5.0  2023-05-02 03:00:00  0.315567
14575      113.0      -6.0     1.0  2023-05-02 03:00:00  0.315567

[14576 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.10570188]
 [-0.10239438]
 [-0.13673222]
 ...
 [-0.82627061]
 [-0.78528026]
 [-0.78528026]]


((11650, 9, 1), (11650, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3887, 3, 1), (5, 3, 1), (486, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
365/365 - 14s - loss: 0.0978 - accuracy: 0.3748 - val_loss: 0.0160 - val_accuracy: 0.0000e+00 - 14s/epoch - 38ms/step
Epoch 2/10
365/365 - 11s - loss: 0.0084 - accuracy: 0.3374 - val_loss: 0.0035 - val_accuracy: 0.0000e+00 - 11s/epoch - 30ms/step
Epoch 3/10
365/365 - 11s - loss: 0.0071 - accuracy: 0.3780 - val_loss: 2.5346e-04 - val_accuracy: 1.0000 - 11s/epoch - 30ms/step
Epoch 4/10
365/365 - 9s - loss: 0.0071 - accuracy: 0.4075 - val_loss: 0.0091 - val_accuracy: 0.0000e+00 - 9s/epoch - 26ms/step
Epoch 5/10
365/365 - 11s - loss: 0.0067 - accuracy: 0.4278 - val_loss: 0.0025 - val_accuracy: 1.0000 - 11s/epoch - 30ms/step
Epoch 6/10
365/365 - 11s - loss: 0.0065 - accuracy: 0.4706 - val_loss: 0.0027 - val_accuracy: 0.0000e+00 - 11s/epoch - 30ms/step
Epoch 7/10
365/365 - 11s - loss: 0.0066 - accuracy: 0.4622 - val_loss: 6.5138e-04 - val_accuracy: 0.0000e+00 - 11s/epoch - 30ms/step
Epoch 8/10
365/365 - 10s - loss: 0.0061 - accuracy: 0.4958 - val_loss: 0.0037 - val_accuracy: 0.000

In [ ]:
test_results = forecast(data,'WPP712')
test_results

1458
1458
(1458, 1) (1458, 1)


,Predictions,Actuals,Date
0,0.369901,0.455060,2023-04-01 19:00:00
1,0.371952,0.447487,2023-04-01 19:00:00
2,0.378484,0.442151,2023-04-01 20:00:00
3,0.454222,0.436815,2023-04-01 20:00:00
4,0.457743,0.431478,2023-04-01 21:00:00
...,...,...,...
1453,0.263272,0.261183,2023-05-02 01:00:00
1454,0.269501,0.280252,2023-05-02 02:00:00
1455,0.300044,0.297910,2023-05-02 02:00:00
1456,0.311757,0.315567,2023-05-02 03:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.02208
corr,0.99839
bias,0.01539
pvalue,0.09285


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp712'+'.h5'
model1.save(saved_model_path)

# WPP713

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh713.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

       longitude  latitude  expver                 time       swh
0          119.0      -7.5     NaN  2022-01-01 00:00:00  0.731053
1          119.0      -7.5     NaN  2022-01-01 01:00:00  0.730850
2          119.0      -7.5     NaN  2022-01-01 02:00:00  0.735531
3          119.0      -7.5     NaN  2022-01-01 03:00:00  0.736549
4          119.0      -7.5     NaN  2022-01-01 04:00:00  0.746388
...          ...       ...     ...                  ...       ...
14571      119.0      -7.5     5.0  2023-05-02 01:00:00  0.423787
14572      119.0      -7.5     5.0  2023-05-02 02:00:00  0.435480
14573      119.0      -7.5     1.0  2023-05-02 02:00:00       NaN
14574      119.0      -7.5     5.0  2023-05-02 03:00:00  0.456748
14575      119.0      -7.5     1.0  2023-05-02 03:00:00       NaN

[14576 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude  expver                 time       swh
0          119.0      -7.5     NaN  2022-01-01 00:00:00  0.731053
1          119.0      -7.5     NaN  2022-01-01 01:00:00  0.730850
2          119.0      -7.5     NaN  2022-01-01 02:00:00  0.735531
3          119.0      -7.5     NaN  2022-01-01 03:00:00  0.736549
4          119.0      -7.5     NaN  2022-01-01 04:00:00  0.746388
...          ...       ...     ...                  ...       ...
14571      119.0      -7.5     5.0  2023-05-02 01:00:00  0.423787
14572      119.0      -7.5     5.0  2023-05-02 02:00:00  0.435480
14573      119.0      -7.5     1.0  2023-05-02 02:00:00  0.446114
14574      119.0      -7.5     5.0  2023-05-02 03:00:00  0.456748
14575      119.0      -7.5     1.0  2023-05-02 03:00:00  0.456748

[14576 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[ 0.32804827]
 [ 0.32754129]
 [ 0.33919772]
 ...
 [-0.38136644]
 [-0.35489164]
 [-0.35489164]]


((11650, 9, 1), (11650, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3887, 3, 1), (5, 3, 1), (486, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
365/365 - 17s - loss: 3.2776 - accuracy: 0.3518 - val_loss: 0.0016 - val_accuracy: 0.2500 - 17s/epoch - 47ms/step
Epoch 2/10
365/365 - 15s - loss: 0.0101 - accuracy: 0.4179 - val_loss: 0.0015 - val_accuracy: 0.0000e+00 - 15s/epoch - 41ms/step
Epoch 3/10
365/365 - 11s - loss: 0.0081 - accuracy: 0.4724 - val_loss: 0.0027 - val_accuracy: 0.0000e+00 - 11s/epoch - 29ms/step
Epoch 4/10
365/365 - 16s - loss: 0.0074 - accuracy: 0.4576 - val_loss: 0.0022 - val_accuracy: 0.2500 - 16s/epoch - 43ms/step
Epoch 5/10
365/365 - 14s - loss: 0.0071 - accuracy: 0.4934 - val_loss: 0.0019 - val_accuracy: 0.5000 - 14s/epoch - 38ms/step
Epoch 6/10
365/365 - 14s - loss: 0.0070 - accuracy: 0.5075 - val_loss: 3.4030e-05 - val_accuracy: 0.7500 - 14s/epoch - 38ms/step
Epoch 7/10
365/365 - 16s - loss: 0.0070 - accuracy: 0.5232 - val_loss: 0.0053 - val_accuracy: 0.2500 - 16s/epoch - 44ms/step
Epoch 8/10
365/365 - 13s - loss: 0.0069 - accuracy: 0.5264 - val_loss: 5.0851e-04 - val_accuracy: 0.0000e+00 - 13

In [ ]:
test_results = forecast(data,'WPP713')
test_results

1458
1458
(1458, 1) (1458, 1)


,Predictions,Actuals,Date
0,0.429807,0.370070,2023-04-01 19:00:00
1,0.438383,0.370815,2023-04-01 19:00:00
2,0.438975,0.371208,2023-04-01 20:00:00
3,0.371269,0.371600,2023-04-01 20:00:00
4,0.372743,0.372738,2023-04-01 21:00:00
...,...,...,...
1453,0.426976,0.423787,2023-05-02 01:00:00
1454,0.425781,0.435480,2023-05-02 02:00:00
1455,0.452662,0.446114,2023-05-02 02:00:00
1456,0.466700,0.456748,2023-05-02 03:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.01903
corr,0.99727
bias,0.01148
pvalue,0.08808


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp713'+'.h5'
model1.save(saved_model_path)

# WPP714

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh714.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

       longitude  latitude  expver                 time       swh
0          127.5      -7.5     NaN  2022-01-01 00:00:00  0.664423
1          127.5      -7.5     NaN  2022-01-01 01:00:00  0.650309
2          127.5      -7.5     NaN  2022-01-01 02:00:00  0.651327
3          127.5      -7.5     NaN  2022-01-01 03:00:00  0.665033
4          127.5      -7.5     NaN  2022-01-01 04:00:00  0.690003
...          ...       ...     ...                  ...       ...
14571      127.5      -7.5     5.0  2023-05-02 01:00:00  0.800478
14572      127.5      -7.5     5.0  2023-05-02 02:00:00  0.801969
14573      127.5      -7.5     1.0  2023-05-02 02:00:00       NaN
14574      127.5      -7.5     5.0  2023-05-02 03:00:00  0.801341
14575      127.5      -7.5     1.0  2023-05-02 03:00:00       NaN

[14576 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude  expver                 time       swh
0          127.5      -7.5     NaN  2022-01-01 00:00:00  0.664423
1          127.5      -7.5     NaN  2022-01-01 01:00:00  0.650309
2          127.5      -7.5     NaN  2022-01-01 02:00:00  0.651327
3          127.5      -7.5     NaN  2022-01-01 03:00:00  0.665033
4          127.5      -7.5     NaN  2022-01-01 04:00:00  0.690003
...          ...       ...     ...                  ...       ...
14571      127.5      -7.5     5.0  2023-05-02 01:00:00  0.800478
14572      127.5      -7.5     5.0  2023-05-02 02:00:00  0.801969
14573      127.5      -7.5     1.0  2023-05-02 02:00:00  0.801655
14574      127.5      -7.5     5.0  2023-05-02 03:00:00  0.801341
14575      127.5      -7.5     1.0  2023-05-02 03:00:00  0.801341

[14576 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.1956182 ]
 [-0.22415981]
 [-0.22210144]
 ...
 [ 0.08190912]
 [ 0.08127438]
 [ 0.08127438]]


((11650, 9, 1), (11650, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3887, 3, 1), (5, 3, 1), (486, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
365/365 - 12s - loss: 0.0587 - accuracy: 0.3494 - val_loss: 0.0034 - val_accuracy: 0.5000 - 12s/epoch - 32ms/step
Epoch 2/10
365/365 - 12s - loss: 0.0077 - accuracy: 0.4381 - val_loss: 0.0030 - val_accuracy: 0.5000 - 12s/epoch - 33ms/step
Epoch 3/10
365/365 - 10s - loss: 0.0069 - accuracy: 0.4858 - val_loss: 8.1415e-04 - val_accuracy: 0.2500 - 10s/epoch - 28ms/step
Epoch 4/10
365/365 - 10s - loss: 0.0063 - accuracy: 0.5069 - val_loss: 0.0017 - val_accuracy: 0.2500 - 10s/epoch - 28ms/step
Epoch 5/10
365/365 - 9s - loss: 0.0063 - accuracy: 0.5042 - val_loss: 2.0994e-04 - val_accuracy: 0.2500 - 9s/epoch - 25ms/step
Epoch 6/10
365/365 - 11s - loss: 0.0060 - accuracy: 0.4904 - val_loss: 0.0015 - val_accuracy: 0.2500 - 11s/epoch - 29ms/step
Epoch 7/10
365/365 - 10s - loss: 0.0058 - accuracy: 0.5126 - val_loss: 3.0677e-04 - val_accuracy: 0.5000 - 10s/epoch - 28ms/step
Epoch 8/10
365/365 - 10s - loss: 0.0056 - accuracy: 0.5180 - val_loss: 3.0807e-04 - val_accuracy: 0.2500 - 10s/epoc

In [ ]:
test_results = forecast(data,'WPP714')
test_results

1458
1458
(1458, 1) (1458, 1)


,Predictions,Actuals,Date
0,0.595563,0.255061,2023-04-01 19:00:00
1,0.577979,0.255297,2023-04-01 19:00:00
2,0.563177,0.261222,2023-04-01 20:00:00
3,0.187899,0.267147,2023-04-01 20:00:00
4,0.176245,0.276682,2023-04-01 21:00:00
...,...,...,...
1453,0.801950,0.800478,2023-05-02 01:00:00
1454,0.802195,0.801969,2023-05-02 02:00:00
1455,0.803223,0.801655,2023-05-02 02:00:00
1456,0.803465,0.801341,2023-05-02 03:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.02347
corr,0.99761
bias,-0.00514
pvalue,0.67360


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp714'+'.h5'
model1.save(saved_model_path)

# WPP715


In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh715.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

       longitude  latitude  expver                 time       swh
0          127.0      -1.5     NaN  2022-01-01 00:00:00  0.589989
1          127.0      -1.5     NaN  2022-01-01 01:00:00  0.580490
2          127.0      -1.5     NaN  2022-01-01 02:00:00  0.570312
3          127.0      -1.5     NaN  2022-01-01 03:00:00  0.558370
4          127.0      -1.5     NaN  2022-01-01 04:00:00  0.553010
...          ...       ...     ...                  ...       ...
14571      127.0      -1.5     5.0  2023-05-02 01:00:00  0.331969
14572      127.0      -1.5     5.0  2023-05-02 02:00:00  0.351510
14573      127.0      -1.5     1.0  2023-05-02 02:00:00       NaN
14574      127.0      -1.5     5.0  2023-05-02 03:00:00  0.365165
14575      127.0      -1.5     1.0  2023-05-02 03:00:00       NaN

[14576 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude  expver                 time       swh
0          127.0      -1.5     NaN  2022-01-01 00:00:00  0.589989
1          127.0      -1.5     NaN  2022-01-01 01:00:00  0.580490
2          127.0      -1.5     NaN  2022-01-01 02:00:00  0.570312
3          127.0      -1.5     NaN  2022-01-01 03:00:00  0.558370
4          127.0      -1.5     NaN  2022-01-01 04:00:00  0.553010
...          ...       ...     ...                  ...       ...
14571      127.0      -1.5     5.0  2023-05-02 01:00:00  0.331969
14572      127.0      -1.5     5.0  2023-05-02 02:00:00  0.351510
14573      127.0      -1.5     1.0  2023-05-02 02:00:00  0.358337
14574      127.0      -1.5     5.0  2023-05-02 03:00:00  0.365165
14575      127.0      -1.5     1.0  2023-05-02 03:00:00  0.365165

[14576 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[ 0.43532255]
 [ 0.39920887]
 [ 0.360516  ]
 ...
 [-0.44535123]
 [-0.41939509]
 [-0.41939509]]


((11650, 9, 1), (11650, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3887, 3, 1), (5, 3, 1), (486, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
365/365 - 13s - loss: 0.0745 - accuracy: 0.3560 - val_loss: 0.0010 - val_accuracy: 0.2500 - 13s/epoch - 36ms/step
Epoch 2/10
365/365 - 11s - loss: 0.0116 - accuracy: 0.4435 - val_loss: 0.0011 - val_accuracy: 0.2500 - 11s/epoch - 30ms/step
Epoch 3/10
365/365 - 10s - loss: 0.0096 - accuracy: 0.4881 - val_loss: 0.0036 - val_accuracy: 0.5000 - 10s/epoch - 29ms/step
Epoch 4/10
365/365 - 9s - loss: 0.0100 - accuracy: 0.4967 - val_loss: 0.0039 - val_accuracy: 0.5000 - 9s/epoch - 25ms/step
Epoch 5/10
365/365 - 10s - loss: 0.0093 - accuracy: 0.5106 - val_loss: 0.0154 - val_accuracy: 0.5000 - 10s/epoch - 29ms/step
Epoch 6/10
365/365 - 11s - loss: 0.0106 - accuracy: 0.5203 - val_loss: 0.0121 - val_accuracy: 0.2500 - 11s/epoch - 29ms/step
Epoch 7/10
365/365 - 10s - loss: 0.0092 - accuracy: 0.5197 - val_loss: 0.0027 - val_accuracy: 0.2500 - 10s/epoch - 26ms/step
Epoch 8/10
365/365 - 10s - loss: 0.0093 - accuracy: 0.5260 - val_loss: 0.0076 - val_accuracy: 0.5000 - 10s/epoch - 27ms/step
Ep

In [ ]:
test_results = forecast(data,'WPP715')
test_results

In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp715'+'.h5'
model1.save(saved_model_path)

# WPP716

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/swh/csv/swh716.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

       longitude  latitude  expver                 time  swh
0          125.0       1.5     NaN  2022-01-01 00:00:00  NaN
1          125.0       1.5     NaN  2022-01-01 01:00:00  NaN
2          125.0       1.5     NaN  2022-01-01 02:00:00  NaN
3          125.0       1.5     NaN  2022-01-01 03:00:00  NaN
4          125.0       1.5     NaN  2022-01-01 04:00:00  NaN
...          ...       ...     ...                  ...  ...
14571      125.0       1.5     5.0  2023-05-02 01:00:00  NaN
14572      125.0       1.5     5.0  2023-05-02 02:00:00  NaN
14573      125.0       1.5     1.0  2023-05-02 02:00:00  NaN
14574      125.0       1.5     5.0  2023-05-02 03:00:00  NaN
14575      125.0       1.5     1.0  2023-05-02 03:00:00  NaN

[14576 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

Percentage of NaN values in the SWH data: 100.00%
There are NaN values in the SWH data.
       longitude  latitude  expver                 time  swh
0          125.0       1.5     NaN  2022-01-01 00:00:00  NaN
1          125.0       1.5     NaN  2022-01-01 01:00:00  NaN
2          125.0       1.5     NaN  2022-01-01 02:00:00  NaN
3          125.0       1.5     NaN  2022-01-01 03:00:00  NaN
4          125.0       1.5     NaN  2022-01-01 04:00:00  NaN
...          ...       ...     ...                  ...  ...
14571      125.0       1.5     5.0  2023-05-02 01:00:00  NaN
14572      125.0       1.5     5.0  2023-05-02 02:00:00  NaN
14573      125.0       1.5     1.0  2023-05-02 02:00:00  NaN
14574      125.0       1.5     5.0  2023-05-02 03:00:00  NaN
14575      125.0       1.5     1.0  2023-05-02 03:00:00  NaN

[14576 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


((11650, 9, 1), (11650, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3887, 3, 1), (5, 3, 1), (486, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10

Reached 99% accuracy so cancelling training!
365/365 - 14s - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000 - 14s/epoch - 38ms/step


In [ ]:
test_results = forecast(data,'WPP716')
test_results

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


1458
1458
(1458, 1) (1458, 1)


,Predictions,Actuals,Date
0,NaN,NaN,2023-04-01 19:00:00
1,NaN,NaN,2023-04-01 19:00:00
2,NaN,NaN,2023-04-01 20:00:00
3,NaN,NaN,2023-04-01 20:00:00
4,NaN,NaN,2023-04-01 21:00:00
...,...,...,...
1453,NaN,NaN,2023-05-02 01:00:00
1454,NaN,NaN,2023-05-02 02:00:00
1455,NaN,NaN,2023-05-02 02:00:00
1456,NaN,NaN,2023-05-02 03:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

ValueError: ignored

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/swh/csv'+'wpp716'+'.h5'
model1.save(saved_model_path)

# WPP717

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh717.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

       longitude  latitude  expver                 time       swh
0          137.0       0.0     NaN  2022-01-01 00:00:00  1.386571
1          137.0       0.0     NaN  2022-01-01 01:00:00  1.379786
2          137.0       0.0     NaN  2022-01-01 02:00:00  1.371033
3          137.0       0.0     NaN  2022-01-01 03:00:00  1.361805
4          137.0       0.0     NaN  2022-01-01 04:00:00  1.354545
...          ...       ...     ...                  ...       ...
14571      137.0       0.0     5.0  2023-05-02 01:00:00  1.267731
14572      137.0       0.0     5.0  2023-05-02 02:00:00  1.265612
14573      137.0       0.0     1.0  2023-05-02 02:00:00       NaN
14574      137.0       0.0     5.0  2023-05-02 03:00:00  1.263337
14575      137.0       0.0     1.0  2023-05-02 03:00:00       NaN

[14576 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude  expver                 time       swh
0          137.0       0.0     NaN  2022-01-01 00:00:00  1.386571
1          137.0       0.0     NaN  2022-01-01 01:00:00  1.379786
2          137.0       0.0     NaN  2022-01-01 02:00:00  1.371033
3          137.0       0.0     NaN  2022-01-01 03:00:00  1.361805
4          137.0       0.0     NaN  2022-01-01 04:00:00  1.354545
...          ...       ...     ...                  ...       ...
14571      137.0       0.0     5.0  2023-05-02 01:00:00  1.267731
14572      137.0       0.0     5.0  2023-05-02 02:00:00  1.265612
14573      137.0       0.0     1.0  2023-05-02 02:00:00  1.264474
14574      137.0       0.0     5.0  2023-05-02 03:00:00  1.263337
14575      137.0       0.0     1.0  2023-05-02 03:00:00  1.263337

[14576 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[ 0.23723141]
 [ 0.21851314]
 [ 0.19436695]
 ...
 [-0.0995914 ]
 [-0.10273061]
 [-0.10273061]]


((11650, 9, 1), (11650, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3887, 3, 1), (5, 3, 1), (486, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
365/365 - 14s - loss: 0.2665 - accuracy: 0.3676 - val_loss: 1.5712e-04 - val_accuracy: 1.0000 - 14s/epoch - 38ms/step
Epoch 2/10
365/365 - 12s - loss: 0.0141 - accuracy: 0.4073 - val_loss: 9.3505e-04 - val_accuracy: 1.0000 - 12s/epoch - 32ms/step
Epoch 3/10
365/365 - 12s - loss: 0.0079 - accuracy: 0.4234 - val_loss: 0.0041 - val_accuracy: 0.0000e+00 - 12s/epoch - 33ms/step
Epoch 4/10
365/365 - 11s - loss: 0.0069 - accuracy: 0.4512 - val_loss: 7.7505e-04 - val_accuracy: 1.0000 - 11s/epoch - 30ms/step
Epoch 5/10
365/365 - 11s - loss: 0.0061 - accuracy: 0.4688 - val_loss: 0.0013 - val_accuracy: 1.0000 - 11s/epoch - 31ms/step
Epoch 6/10
365/365 - 12s - loss: 0.0056 - accuracy: 0.4922 - val_loss: 1.0777e-04 - val_accuracy: 1.0000 - 12s/epoch - 33ms/step
Epoch 7/10
365/365 - 12s - loss: 0.0054 - accuracy: 0.5087 - val_loss: 8.1989e-05 - val_accuracy: 1.0000 - 12s/epoch - 33ms/step
Epoch 8/10
365/365 - 12s - loss: 0.0052 - accuracy: 0.5114 - val_loss: 4.3405e-04 - val_accuracy: 1.0

In [ ]:
test_results = forecast(data,'WPP717')
test_results

In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp717'+'.h5'
model1.save(saved_model_path)

# WPP718

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv/swh718.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# sorted_df = df.sort_values('Age')
print(data)
# data = preparedata(daerah)

### Interpolate the missing values

In [ ]:
data['swh'] = data['swh'].interpolate(method='linear', limit_direction='both')
# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['swh'].isna().any()
nan_percentage = (data['swh'].isna().sum() / len(data['swh'])) * 100

### Check if there are any nan values

In [ ]:


# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
train.shape, val.shape, test.shape

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

In [ ]:
test_results = forecast(data,'WPP718')
test_results

In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/Machine_Learning/data/weather_and_marine_condition/swh/csv'+'modelswhwpp718'+'.h5'
model1.save(saved_model_path)